這份 Notebook 示範 Prompt Engineering 評估過程

* 使用 zero-shot 和 (random) few-shot prompting
* 有標準答案的評估
* 沒有標準答案的評估

首先設定 OpenAI API Key 變數

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [2]:
import requests
import json
from pprint import pp

def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300, format_type=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if format_type:
    payload["response_format"] =  { "type": format_type }

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

# Prompt Engineering

以下示範做一個書籍分類和推薦，以及評估

In [ ]:
# 這是練習用的 dataset，有 title, description 以及標註的正確分類 category
url = "https://ihower.tw/data/books-dataset-200.csv"
response = requests.get(url)

In [ ]:
import pandas as pd
from io import StringIO
pd.read_csv(StringIO(response.text))

,title,description,category,tags
0,統計學：重點觀念與題解 (下) 3/e (適用: 大學用書．高普考．商管所),1. 本書涵蓋範圍既深且廣，難度可輕鬆應付頂尖大學各系所。\r\n2. 本書有各種經濟學與財...,機率統計學 Probability-and-statistics,統計學
1,新時代 乙級數位電子術科含學科題庫實作寶典 - 使用 KiCad/Quartus II / ...,1.依勞動部最新公告試題編而成，專爲有意參加「數位電子乙級技術士技能檢定」之人士編寫，並適合...,技能檢定乙級 Skilltest-b,"技能檢定乙級,考試認證"
2,樂高 SPIKE 機器人創意專題實作 -- 使用 LEGO Education SPIKE ...,1.透過「螺旋式教學」提供一套具有邏輯先後順序教材，由具體到抽象，由簡單到複雜的創客教材。\...,機器人製作 Robots,"樂高,機器人"
3,AI 生成時代：從 ChatGPT 到繪圖、音樂、影片，利用智能創作自我加值、簡化工作，成為...,未來2～5年將是AI 生成的時代\r\n史上用戶最快破億、開啟產業新動能的技術，你跟上了嗎？...,人工智慧,ChatGPT
4,都問 AI 吧！ChatGPT 上手的第一本書,"""寫文案、腳本，編教材，做專案，作業輔助\r\n\r\n都可以問AI，但是——\r\n\r\...",ChatGPT,ChatGPT
...,...,...,...,...
194,Autodesk Civil 3D 2024 from Start to Finish: A...,Master Autodesk Civil 3D 2023 to develop real ...,AutoCAD,AutoDESK
195,Network Programming and Automation Essentials:...,Unleash the power of automation by mastering n...,Go 程式語言,"Go,Python"
196,Internet of Things for Smart Buildings: Levera...,Harness the full potential of IoT in your buil...,物聯網 IoT,物聯網IoT
197,AI and Business Rule Engines for Excel Power U...,A power-packed manual to enhance your decision...,Excel,"Excel,人工智慧"


In [ ]:
import csv
import random

f = StringIO(response.text)
reader = csv.DictReader(f)
data = list(reader)

# 隨機選擇20筆資料來做實驗就好了
selected_data = random.sample(data, 20)

# 印出選擇的資料
for row in selected_data:
    print(row)

{'title': 'The Service-Oriented Enterprise: Learn Enterprise Architecture and Its Viable Services', 'description': "A service-oriented architecture is fundamental to many new IT applications from web development to social software and cloud computing. The same principles can be applied to every aspect of the service-oriented enterprise - not just in IT. In this book you'll explore how an enterprise architecture and viable services can link together to create a simpler yet far more powerful view of the enterprise as a dynamic unified whole. You can use the ideas principles and methods described here in busines", 'category': 'SOA', 'tags': 'SOA'}
{'title': '簡報處理 PowerPoint 2019 一切搞定', 'description': '‧全新2019學習方式\r\n‧紮實簡報設計的能力\r\n‧重視觀念與範例實作\r\n‧內容貼近生活應用面\r\n\r\n■本書以教學為導向，設計循序漸進的範例實作與引導，輕鬆學習PowerPoint 2019必備功能與簡報應用技巧。\r\n\r\n■透過範例操作建立正確的PowerPoint 2019簡報與觀念，靈活運用在簡報設計上，建立紮實且穩固的基礎，讓學習無障礙，也更貼近生活應用。\r\n\r\n‧輕鬆認識PowerPoint 2019：介紹操作環境與介面運用。\r\n‧讓簡報改頭換面的神奇魔法：標題、文字樣式與色彩配置。\r\n‧製作班級簡報：插入圖片格式設定與裁剪圖

In [ ]:
df = pd.DataFrame(selected_data)

## 逐筆進行分類

In [ ]:
for index, row in df.iterrows():
    messages = [
        {
          "role": "system",
          "content": "請根據以下書籍資訊，選擇最適合的技能分類。只需要回傳完整分類名稱，不要回傳其他文字，分類只能在以下選一個: 3D 列印,Arduino,C 程式語言,C++ 程式語言,CI/CD,ChatGPT,Computer Vision,Computer-Science,Computer-architecture,Data Science,Data-visualization,DeepLearning,Domain-Driven Design,Engineer self-growth,Go 程式語言,Information-management,Java,Linux,Machine Learning,Operating-system,Python,R 語言,Reinforcement,UI/UX,Version Control,Visual C#,人工智慧,化學 Chemistry,區塊鏈與金融科技,地理資訊系統 Gis,大數據 Big-data,微軟技術,搜尋引擎優化 SEO,數學,機器人製作 Robots,物聯網 IoT,管理與領導 Management-leadership,網站開發,網頁設計,職涯發展,行動通訊 Mobile-communication,行銷/網路行銷 Marketing,資料庫,資訊安全,軟體工程,軟體架構,軟體測試,雲端運算,電力電子 Power-electronics,電機學 Electric-machinery,電路學 Electric-circuits"        },
        {
            "role": "user",
            "content": f"書名: {row['title']} \n 描述: {row['description']}"
        }
    ]
    response = get_completion(messages, model="gpt-3.5-turbo")
    df.at[index, 'predict'] = response
    # pp(response)

df

,title,description,category,tags,predict
0,The Service-Oriented Enterprise: Learn Enterpr...,A service-oriented architecture is fundamental...,SOA,SOA,軟體架構
1,簡報處理 PowerPoint 2019 一切搞定,‧全新2019學習方式\r\n‧紮實簡報設計的能力\r\n‧重視觀念與範例實作\r\n‧內容...,PowerPoint,"PowerPoint,簡報製作",UI/UX
2,5G承載網建設與維護,本書圍繞快速發展的移動通信技術—5G承載網技術進行編寫，從5G承載網組網架構和關鍵技術切入，...,5G,5G,行動通訊 Mobile-communication
3,Available-To-Promise with SAP S/4hana: Advance...,See how SAP S/4HANA for advanced ATP improves ...,SAP,"SAP,網路管理 MIS",資訊管理
4,Vue.js 3.x 從入門到精通 (視頻教學版),本書通過應用示例和綜合案例的講解與演練，使讀者快速掌握Vue.js 3.x編程知識，提高使用...,Vue.js,Vue.js,網站開發
5,Formal Verification of Floating-Point Hardware...,This is the first book to focus on the problem...,Algorithms-data-structures,演算法,電路學 Electric-circuits
6,都問 AI 吧！ChatGPT 上手的第一本書,"""寫文案、腳本，編教材，做專案，作業輔助\r\n\r\n都可以問AI，但是——\r\n\r\...",ChatGPT,ChatGPT,ChatGPT
7,Introduction to Photonics and Laser Physics wi...,Lasers are ubiquitous-from deep space communic...,光電子學 Photonics,"光電子學,物理學",電路學 Electric-circuits
8,Ai-Powered Iot for Covid-19,The Internet of Things (IoT) has made revoluti...,物聯網 IoT,物聯網IoT,物聯網 IoT
9,SSM + Vue.js 3 全棧開發實戰,SSM（Spring、Spring MVC和MyBatis）和Vue.js 3是當前使用廣泛...,Vue.js,Vue.js,網站開發


## 逐筆寫個推薦的話

In [ ]:
for index, row in df.iterrows():
    messages = [
        {
          "role": "system",
          "content": "根據以下書籍資料，如果我想推薦給別人，應該要說什麼?"
        },
        {
            "role": "user",
            "content": row["title"] + row["description"]
        }
    ]
    response = get_completion(messages)
    df.at[index, 'recommend'] = response

In [ ]:
df[["title", "recommend"]]

,title,recommend
0,The Service-Oriented Enterprise: Learn Enterpr...,如果你想推薦這本書給別人，你可以說：這本書《The Service-Oriented Ent...
1,簡報處理 PowerPoint 2019 一切搞定,這本書提供了很全面的PowerPoint 2019學習方式，強調紮實的簡報設計能力，並且重視...
2,5G承載網建設與維護,這本書《5G承載網建設與維護》是一本針對快速發展的移動通信技術5G承載網的技術進行編寫的實用...
3,Available-To-Promise with SAP S/4hana: Advance...,我推薦這本書給你，因為它提供了關於如何使用SAP S/4HANA進行高級ATP的詳細資訊。這...
4,Vue.js 3.x 從入門到精通 (視頻教學版),我會推薦這本書給想要學習Vue.js 3.x的人，因為它提供了豐富的教學資源，包括示例源碼、...
5,Formal Verification of Floating-Point Hardware...,這本書是專注於通過數學方法確保浮點硬件設計正確性的第一本書。《Formal Verifica...
6,都問 AI 吧！ChatGPT 上手的第一本書,這本書是針對對新科學技術好奇且有熱情的一般大眾、科技愛好者、對AI和NLP感興趣、不懂程式語...
7,Introduction to Photonics and Laser Physics wi...,這本書《Introduction to Photonics and Laser Physic...
8,Ai-Powered Iot for Covid-19,如果你想推薦這本書給別人，你可以說：這本書探討了人工智慧和物聯網在COVID-19大流行中的...
9,SSM + Vue.js 3 全棧開發實戰,這本書《SSM + Vue.js 3 全棧開發實戰》是一本循序漸進地介紹SSM（Spring...


## 進行評估 Evaluation

可分為三種情況:

* 有標準答案，可以算準確率
* 有參考答案，可以比較兩者相似度: 讓 LLM 幫你評分有多相似
* 沒有參考答案，只有評分標準: 讓 LLM 幫你打分數

### 計算準確率，和資料集的標注分類 category 相比

In [ ]:
# 計算兩個欄位一致的筆數
correct_predictions = (df['category'] == df['predict']).sum()

# 計算準確率
accuracy = correct_predictions / len(df)


print(f"準確率: {accuracy:.2%}")

準確率: 20.00%


## (Random) Few-shot prompting

換一種 prompting 方式實驗看看，我們隨機挑 *10* 個範例當作 few-shot

In [ ]:
selected_data = random.sample(data, 10)
examples = ''

for row in selected_data:
    examples = examples + f"""
書名: {row["title"]}
描述: ```{row["description"]}```
分類: {row["category"]}
----
"""

In [ ]:
for index, row in df.iterrows():
    messages = [
        {
          "role": "system",
          "content": "請根據以下書籍資訊，選擇最適合的技能分類。只需要回傳完整分類名稱，不要回傳其他文字，分類只能在以下選一個: 3D 列印,Arduino,C 程式語言,C++ 程式語言,CI/CD,ChatGPT,Computer Vision,Computer-Science,Computer-architecture,Data Science,Data-visualization,DeepLearning,Domain-Driven Design,Engineer self-growth,Go 程式語言,Information-management,Java,Linux,Machine Learning,Operating-system,Python,R 語言,Reinforcement,UI/UX,Version Control,Visual C#,人工智慧,化學 Chemistry,區塊鏈與金融科技,地理資訊系統 Gis,大數據 Big-data,微軟技術,搜尋引擎優化 SEO,數學,機器人製作 Robots,物聯網 IoT,管理與領導 Management-leadership,網站開發,網頁設計,職涯發展,行動通訊 Mobile-communication,行銷/網路行銷 Marketing,資料庫,資訊安全,軟體工程,軟體架構,軟體測試,雲端運算,電力電子 Power-electronics,電機學 Electric-machinery,電路學 Electric-circuits"
        },
        {
          "role": "user",
          "content": examples
        },
        {
          "role": "user",
          "content": f"書名: {row['title']} \n 描述: {row['description']}\n 分類:"
        }
    ]
    response = get_completion(messages, model="gpt-3.5-turbo")
    df.at[index, 'predict2'] = response


In [ ]:
df

,title,description,category,tags,predict,recommend,predict2
0,The Service-Oriented Enterprise: Learn Enterpr...,A service-oriented architecture is fundamental...,SOA,SOA,軟體架構,如果你想推薦這本書給別人，你可以說：這本書《The Service-Oriented Ent...,Domain-Driven Design
1,簡報處理 PowerPoint 2019 一切搞定,‧全新2019學習方式\r\n‧紮實簡報設計的能力\r\n‧重視觀念與範例實作\r\n‧內容...,PowerPoint,"PowerPoint,簡報製作",UI/UX,這本書提供了很全面的PowerPoint 2019學習方式，強調紮實的簡報設計能力，並且重視...,網站開發
2,5G承載網建設與維護,本書圍繞快速發展的移動通信技術—5G承載網技術進行編寫，從5G承載網組網架構和關鍵技術切入，...,5G,5G,行動通訊 Mobile-communication,這本書《5G承載網建設與維護》是一本針對快速發展的移動通信技術5G承載網的技術進行編寫的實用...,行動通訊 Mobile-communication
3,Available-To-Promise with SAP S/4hana: Advance...,See how SAP S/4HANA for advanced ATP improves ...,SAP,"SAP,網路管理 MIS",資訊管理,我推薦這本書給你，因為它提供了關於如何使用SAP S/4HANA進行高級ATP的詳細資訊。這...,資訊管理
4,Vue.js 3.x 從入門到精通 (視頻教學版),本書通過應用示例和綜合案例的講解與演練，使讀者快速掌握Vue.js 3.x編程知識，提高使用...,Vue.js,Vue.js,網站開發,我會推薦這本書給想要學習Vue.js 3.x的人，因為它提供了豐富的教學資源，包括示例源碼、...,網站開發
5,Formal Verification of Floating-Point Hardware...,This is the first book to focus on the problem...,Algorithms-data-structures,演算法,電路學 Electric-circuits,這本書是專注於通過數學方法確保浮點硬件設計正確性的第一本書。《Formal Verifica...,電路學 Electric-circuits
6,都問 AI 吧！ChatGPT 上手的第一本書,"""寫文案、腳本，編教材，做專案，作業輔助\r\n\r\n都可以問AI，但是——\r\n\r\...",ChatGPT,ChatGPT,ChatGPT,這本書是針對對新科學技術好奇且有熱情的一般大眾、科技愛好者、對AI和NLP感興趣、不懂程式語...,ChatGPT
7,Introduction to Photonics and Laser Physics wi...,Lasers are ubiquitous-from deep space communic...,光電子學 Photonics,"光電子學,物理學",電路學 Electric-circuits,這本書《Introduction to Photonics and Laser Physic...,光電子學 Photonics
8,Ai-Powered Iot for Covid-19,The Internet of Things (IoT) has made revoluti...,物聯網 IoT,物聯網IoT,物聯網 IoT,如果你想推薦這本書給別人，你可以說：這本書探討了人工智慧和物聯網在COVID-19大流行中的...,物聯網 IoT
9,SSM + Vue.js 3 全棧開發實戰,SSM（Spring、Spring MVC和MyBatis）和Vue.js 3是當前使用廣泛...,Vue.js,Vue.js,網站開發,這本書《SSM + Vue.js 3 全棧開發實戰》是一本循序漸進地介紹SSM（Spring...,網站開發


In [ ]:
# 計算兩個欄位一致的筆數
correct_predictions2 = (df['category'] == df['predict2']).sum()

# 計算準確率
accuracy2 = correct_predictions2 / len(df)


print(f"準確率: {accuracy2:.2%}")

準確率: 25.00%


* 模型可換成用 gpt-4-turbo-preview 看看，準確率可以大幅上升。只是成本高很多.... XD
* Part 2 課程還會教一招 dynamic few-shot prompting 技巧，聰明的找出相似的範例
* Part 3 課程還會教如何用 function calling 來做這題，也可以提高準確率

## 評估產生的推薦

prompt 參考出處: https://spellbook.scale.com/

注意: 評分最好也設計 CoT，先輸出一些理由解釋，最後才輸出分數

In [ ]:
criteria = "答案必須 1. 需用台灣繁體中文(除了書名可用英文) 2.足夠吸引人瀏覽 3.和書籍內容相關 4.順口能被記住"

for index, row in df.iterrows():
    question = "根據以下書籍資料，如果我想推薦給別人，應該要說什麼? \n" + row["title"] + " \n" + row["description"]
    answer = row["recommend"]

    prompt = f"""
    [Question]

    {question}

    [The Start of Assistant's Answer]

    {answer}

    [The End of Assistant's Answer]

    [Criteria]

    {criteria}

    [System]

    We would like to request your feedback on the performance of an AI assistant in response to the user question displayed above.
    Please rate their response according to the criteria defined above. Do not consider any other criteria.
    The assistant receives an overall score on a scale of 1 to 5, where a higher score indicates better adherence to the criteria.
    Please provide a explanation of your evaluation, avoiding any potential bias, using taiwanese chinese.
    Then output only one value indicating the scores for the assistant.

    [Output JSON example]
    {{
      "reasoning": "a explanation of your evaluation>"
      "score": "integer"
    }}
    """

    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    response = get_completion(messages, model="gpt-4-1106-preview", max_tokens=1000, format_type="json_object") # 評估需要推理得用 gpt-4 效果才好
    eval_data = json.loads(response)

    df.at[index, 'recommend_score'] = eval_data["score"]
    df.at[index, 'recommend_reason'] = eval_data["reasoning"]

In [ ]:
df[["title", "recommend", "recommend_score", "recommend_reason"]]

,title,recommend,recommend_score,recommend_reason
0,The Service-Oriented Enterprise: Learn Enterpr...,如果你想推薦這本書給別人，你可以說：這本書《The Service-Oriented Ent...,3.0,根據評分標準，助理的回答使用了台灣繁體中文，並且保留了書名的英文原文，符合第一點要求。回答內...
1,簡報處理 PowerPoint 2019 一切搞定,這本書提供了很全面的PowerPoint 2019學習方式，強調紮實的簡報設計能力，並且重視...,5.0,AI助理的回答符合了提問的要求，使用了台灣繁體中文進行回答，並且內容與書籍相關，足夠吸引人瀏...
2,5G承載網建設與維護,這本書《5G承載網建設與維護》是一本針對快速發展的移動通信技術5G承載網的技術進行編寫的實用...,3.0,AI助理的回答符合了提問者的要求，使用了台灣繁體中文來進行回答，並且提供了書籍的主要內容和適...
3,Available-To-Promise with SAP S/4hana: Advance...,我推薦這本書給你，因為它提供了關於如何使用SAP S/4HANA進行高級ATP的詳細資訊。這...,3.0,根據所提供的評分標準，助理的回答使用了台灣繁體中文，並且保留了書名的英文原文，符合第一條標準...
4,Vue.js 3.x 從入門到精通 (視頻教學版),我會推薦這本書給想要學習Vue.js 3.x的人，因為它提供了豐富的教學資源，包括示例源碼、...,4.0,AI助理的回答符合了提問的要求，使用了台灣繁體中文來進行回答，並且書名使用了英文。回答中提到...
5,Formal Verification of Floating-Point Hardware...,這本書是專注於通過數學方法確保浮點硬件設計正確性的第一本書。《Formal Verifica...,5.0,根據所提供的評分標準，AI助理的回答基本上符合了所有的要求。首先，答案使用了台灣繁體中文來進...
6,都問 AI 吧！ChatGPT 上手的第一本書,這本書是針對對新科學技術好奇且有熱情的一般大眾、科技愛好者、對AI和NLP感興趣、不懂程式語...,3.0,AI助理的回答符合了提問的要求，使用了台灣繁體中文來進行回答，並且書名以英文表示，符合第一條...
7,Introduction to Photonics and Laser Physics wi...,這本書《Introduction to Photonics and Laser Physic...,3,根據評分標準，助理的回答並未完全使用台灣繁體中文，書名除外的部分也使用了英文，這不符合第一條...
8,Ai-Powered Iot for Covid-19,如果你想推薦這本書給別人，你可以說：這本書探討了人工智慧和物聯網在COVID-19大流行中的...,5,AI助理的回答符合了提問的所有標準。首先，回答使用了台灣繁體中文，並且保留了書名的英文原文。...
9,SSM + Vue.js 3 全棧開發實戰,這本書《SSM + Vue.js 3 全棧開發實戰》是一本循序漸進地介紹SSM（Spring...,4,AI助理的回答符合了提問的要求，使用了台灣繁體中文來進行回答，並且將書名以英文表示。回答中提...


# 更多評估 Prompt 範例

1. 在 Building Systems with the ChatGPT API 課程中，也有一個評估範例 Prompt，詳見: Evaluation Part 2 評估(當沒有標準答案時):
https://ihower.tw/notes/%E6%8A%80%E8%A1%93%E7%AD%86%E8%A8%98-AI/OpenAI/Building+Systems+with+the+ChatGPT+API#10.+Evaluation+Part+2+%E8%A9%95%E4%BC%B0(%E7%95%B6%E6%B2%92%E6%9C%89%E6%A8%99%E6%BA%96%E7%AD%94%E6%A1%88%E6%99%82)

2. 在 GPT Best Practies 中的 Tactic: Evaluate model outputs with reference to gold-standard answers: https://platform.openai.com/docs/guides/prompt-engineering/strategy-test-changes-systematically

3. 在 https://ihower.tw/notes/%E6%8A%80%E8%A1%93%E7%AD%86%E8%A8%98-AI/2023/The+New+Stack+and+Ops+for+AI+(OpenAI+DevDay) 演講中，有個 Example scorescard for model-graded evals